In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize as op
import math
import torch
import re

In [2]:
data = pandas.read_csv('./data/Power_hard_sigmoid.csv')
data_split = data.iloc[:, 1:]
data_split

,"power (L=0.00027,R=5100000,w=0.0006) Y","power (L=0.000155,R=7550000,w=0.0004) Y","power (L=0.000385,R=2650000,w=0.0008) Y","power (L=0.000328,R=3875000,w=0.0005) Y","power (L=9.8e-05,R=8775000,w=0.0009) Y","power (L=0.000442,R=6325000,w=0.0003) Y","power (L=0.000212,R=1425000,w=0.0007) Y","power (L=0.000471,R=2037500,w=0.00045) Y","power (L=0.000241,R=6937500,w=0.00085) Y","power (L=0.000356,R=9387500,w=0.00025) Y",...,"power (L=6.1e-05,R=1469861,w=0.000787) Y","power (L=0.00032,R=2082361,w=0.000437) Y","power (L=9e-05,R=6982361,w=0.000837) Y","power (L=0.000435,R=9432361,w=0.000237) Y","power (L=0.000205,R=4532361,w=0.000637) Y","power (L=0.000147,R=3307361,w=0.000337) Y","power (L=0.000377,R=8207361,w=0.000737) Y","power (L=0.000262,R=5757361,w=0.000537) Y","power (L=0.000492,R=857361,w=0.000937) Y","power (L=0.000144,R=735938,w=0.000944) Y"
0,1.150000e-39,2.300000e-39,5.240000e-40,4.120000e-40,3.390000e-38,1.330000e-40,7.110000e-40,8.510000e-41,3.950000e-39,2.120000e-40,...,1.120000e-38,1.780000e-40,2.760000e-38,1.280000e-40,2.000000e-39,7.960000e-40,1.440000e-39,1.110000e-39,1.420000e-40,1.450000e-39
1,3.860000e-39,7.700000e-39,1.750000e-39,1.380000e-39,1.130000e-37,4.460000e-40,2.380000e-39,2.850000e-40,1.320000e-38,7.090000e-40,...,3.750000e-38,5.950000e-40,9.250000e-38,4.290000e-40,6.700000e-39,2.660000e-39,4.810000e-39,3.710000e-39,4.760000e-40,4.850000e-39
2,6.560000e-39,1.310000e-38,2.980000e-39,2.350000e-39,1.930000e-37,7.600000e-40,4.050000e-39,4.850000e-40,2.250000e-38,1.210000e-39,...,6.380000e-38,1.010000e-39,1.570000e-37,7.300000e-40,1.140000e-38,4.530000e-39,8.180000e-39,6.300000e-39,8.110000e-40,8.240000e-39
3,9.270000e-39,1.850000e-38,4.210000e-39,3.310000e-39,2.720000e-37,1.070000e-39,5.720000e-39,6.850000e-40,3.180000e-38,1.700000e-39,...,9.010000e-38,1.430000e-39,2.220000e-37,1.030000e-39,1.610000e-38,6.400000e-39,1.150000e-38,8.900000e-39,1.140000e-39,1.160000e-38
4,1.200000e-38,2.390000e-38,5.440000e-39,4.280000e-39,3.520000e-37,1.390000e-39,7.390000e-39,8.840000e-40,4.100000e-38,2.200000e-39,...,1.160000e-37,1.850000e-39,2.870000e-37,1.330000e-39,2.080000e-38,8.270000e-39,1.490000e-38,1.150000e-38,1.480000e-39,1.500000e-38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,1.950000e-07,1.320000e-07,3.750000e-07,2.570000e-07,1.140000e-07,1.570000e-07,6.970000e-07,4.820000e-07,1.440000e-07,1.060000e-07,...,6.790000e-07,4.740000e-07,1.430000e-07,1.050000e-07,2.200000e-07,3.010000e-07,1.220000e-07,1.730000e-07,1.140000e-06,1.350000e-06
397,1.950000e-07,1.320000e-07,3.750000e-07,2.570000e-07,1.140000e-07,1.570000e-07,6.970000e-07,4.820000e-07,1.440000e-07,1.060000e-07,...,6.790000e-07,4.740000e-07,1.430000e-07,1.050000e-07,2.200000e-07,3.010000e-07,1.220000e-07,1.730000e-07,1.140000e-06,1.350000e-06
398,1.950000e-07,1.320000e-07,3.750000e-07,2.570000e-07,1.140000e-07,1.570000e-07,6.970000e-07,4.820000e-07,1.440000e-07,1.060000e-07,...,6.790000e-07,4.750000e-07,1.430000e-07,1.050000e-07,2.200000e-07,3.010000e-07,1.220000e-07,1.730000e-07,1.140000e-06,1.350000e-06
399,1.950000e-07,1.320000e-07,3.750000e-07,2.570000e-07,1.140000e-07,1.570000e-07,6.970000e-07,4.820000e-07,1.440000e-07,1.060000e-07,...,6.790000e-07,4.750000e-07,1.430000e-07,1.050000e-07,2.200000e-07,3.010000e-07,1.220000e-07,1.730000e-07,1.140000e-06,1.350000e-06


In [3]:
datalist=data_split.columns.values.tolist()
print(datalist)
print(len(datalist))

['power (L=0.00027,R=5100000,w=0.0006) Y', 'power (L=0.000155,R=7550000,w=0.0004) Y', 'power (L=0.000385,R=2650000,w=0.0008) Y', 'power (L=0.000328,R=3875000,w=0.0005) Y', 'power (L=9.8e-05,R=8775000,w=0.0009) Y', 'power (L=0.000442,R=6325000,w=0.0003) Y', 'power (L=0.000212,R=1425000,w=0.0007) Y', 'power (L=0.000471,R=2037500,w=0.00045) Y', 'power (L=0.000241,R=6937500,w=0.00085) Y', 'power (L=0.000356,R=9387500,w=0.00025) Y', 'power (L=0.000126,R=4487500,w=0.00065) Y', 'power (L=0.000184,R=3262500,w=0.00035) Y', 'power (L=0.000414,R=8162500,w=0.00075) Y', 'power (L=6.9e-05,R=5712500,w=0.00055) Y', 'power (L=0.000299,R=812500,w=0.00095) Y', 'power (L=0.000256,R=1118750,w=0.000575) Y', 'power (L=0.000486,R=6018750,w=0.000975) Y', 'power (L=0.000141,R=8468750,w=0.000375) Y', 'power (L=0.000371,R=3568750,w=0.000775) Y', 'power (L=0.000428,R=4793750,w=0.000275) Y', 'power (L=0.000198,R=9693750,w=0.000675) Y', 'power (L=0.000313,R=7243750,w=0.000475) Y', 'power (L=8.3e-05,R=2343750,w=0.000

In [4]:
def extract_values(data_list):
    patterns = [
        r"R=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",   # Pattern for R
        r"w=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",   # Pattern for w
        r"L=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))"    # Pattern for L
    ]
    
    matrix = []
    
    for data in data_list:
        row = []
        for pattern in patterns:
            match = re.search(pattern, data)
            if match:
                value = match.group(1)
                if 'e' in value or '.' in value: 
                    row.append(float(value))
                else:
                    row.append(int(value))
            else:
                row.append(None)
        matrix.append(row)
    return matrix

result = extract_values(datalist)

In [5]:
result_temp = torch.tensor(result)
result_temp.max(0)[0]

tensor([9.9988e+06, 1.0000e-03, 5.0000e-04])

In [6]:
result_temp.min(0)[0]

tensor([2.0120e+05, 2.0000e-04, 4.0000e-05])

In [7]:
c = torch.tensor(result)
c.shape

torch.Size([9999, 3])

In [8]:
torch.save(result_temp, './data/hard_sigmoid_RT.data')

In [9]:
r = result_temp[:, 0].view(-1,1)
w = result_temp[:, 1].view(-1,1)
l = result_temp[:, 2].view(-1,1)
ratio = w/l

r.shape, w.shape, l.shape, ratio.shape

(torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]))

In [10]:
C = torch.cat([r, w, l, ratio], dim=1)
C.shape

torch.Size([9999, 4])

In [11]:
torch.save(C, './data/hard_sigmoid_RT_ratio.data')